In [2]:
import pandas as pd
import numpy as np

# Merge Data (11_22)

##### Merges Voting data from 2012, 2016, & 2020 with Census data for Age, and other demographic files.
##### States: AZ, FL, GA, MI, NC, OH, PA. WI

### File Paths

Only need to change files paths if there are new files

In [3]:
#Output Path
output_file_path = '../Data/Merged_Dataset.csv'

#Voting Data
#Contains 2012, 2016, & 2020 data
voting_data_path = '../Data/Raw Data Files/vote_data_2012-2020.csv'

#Age Data
age_data_path = '../Data/Raw Data Files/US_Census_Pop_Data.csv'

#Demographic data
demo_2012_path = '../Data/census_data/2012_demographic.csv'
demo_2016_path = '../Data/census_data/2016_demographic.csv'
demo_2020_path = '../Data/census_data/2018_demographic.csv'

#State Code Mapping File
state_abbrev_path = '../Data/census_data/state_abbrev.csv'

#FIPS Geo_ID Mapping File
fips_path = '../Data/census_data/geo_ids.csv'

Run Rest of cells to get final csv

#### Voting Data

In [4]:
#Read in Voting Data
voting_data = pd.read_csv(voting_data_path)

#Manully map two counties (There is a mismatch in spelling)
voting_data['county'] = voting_data['county'].replace('GD. TRAVERSE COUNTY',
                                                      'Grand Traverse County')
voting_data['county'] = voting_data['county'].replace('ST.CROIX COUNTY',
                                                      'ST. CROIX COUNTY')
#Ensure all counties are upper case 
voting_data['county'] = voting_data['county'].str.upper()

#Remove commas from voting data
voting_data[['total','dem','gop']] = voting_data[['total',
                                                  'dem',
                                                  'gop']].replace(',','', regex=True)
#Convert Votes Data Type
voting_data[['total','dem','gop']] = voting_data[['total',
                                                  'dem',
                                                  'gop']].astype(int)

#Convert Year to String
voting_data['year'] = voting_data['year'].astype(str)

#Remove Duplicates (Sanity Check)
voting_data.drop_duplicates(inplace = True)

#Print Shape
voting_data.shape

(1953, 8)

In [5]:
#Read in State mapping file
state_abbrv = pd.read_csv(state_abbrev_path)
#Clean data
state_abbrv['Code'] = state_abbrv['Code'].str.upper()
state_abbrv['Code'] = state_abbrv['Code'].str.strip()

#### Age Data

In [6]:
#Read in Age Dataset
age_combined_csv = pd.read_csv(age_data_path)
# Merge Age with state abbrev
age_state_abbrv = age_combined_csv.merge(state_abbrv[['State','Code']],
                                         left_on = 'STNAME',
                                         right_on = 'State')
#Set 2019 data as 2020
age_state_abbrv['YEAR'] = age_state_abbrv['YEAR'].replace('2019','2020')
#Only 2012,2016,2020 data
age_state_abbrv = age_state_abbrv[age_state_abbrv['YEAR'].isin(['2012','2016','2020'])]

#Map Two Counties Correctly: Grand Traverse County
age_state_abbrv['CTYNAME'] = age_state_abbrv['CTYNAME'].replace('GD. TRAVERSE COUNTY',
                                                                'GRAND TRAVERSE COUNTY')
age_state_abbrv['CTYNAME'] = age_state_abbrv['CTYNAME'].replace('ST.CROIX COUNTY',
                                                                'ST. CROIX COUNTY')
#Set Counties to uppercase
age_state_abbrv['CTYNAME'] = age_state_abbrv['CTYNAME'].str.upper()

#Drop extra columns
age_state_abbrv.drop(['SUMLEV',
                      'STATE',
                      'COUNTY',
                      'STNAME',
                      'State'],
                     axis = 1,
                     inplace = True) 
#Drop Any Duplicates
age_state_abbrv.drop_duplicates(inplace = True)
#Sanity Check that all the years are correct
print(age_state_abbrv[~age_state_abbrv['YEAR'].isin(['2012','2016','2020'])].shape)
print(age_state_abbrv.shape)

(0, 93)
(1953, 93)


#### Ssanity Check

In [7]:
# Confirm Age and Voting have the same number of counties
print("Number of voting data counties: {}".format(voting_data.shape[0]))
print("Number of counties in age data: {}".format(age_state_abbrv.shape[0]))

Number of voting data counties: 1953
Number of counties in age data: 1953


In [8]:
#Merge age and voting data
df_harvard = pd.merge(voting_data,
                      age_state_abbrv,
                      how = 'inner',
                      left_on = ['year','state_po','county'],
                      right_on = ['YEAR','Code','CTYNAME'])

In [9]:
#All Counties Merged
df_harvard.shape

(1953, 101)

#### Santiy Check 

In [10]:
# Number of Null Values
df_harvard.isnull().sum().sum()

0

#### Demographic Data

In [11]:
#Read in census datea
census_2012 = pd.read_csv(demo_2012_path)
census_2016 = pd.read_csv(demo_2016_path)
census_2018 = pd.read_csv(demo_2020_path)
#Load Mapping File
FIPS_map = pd.read_csv(fips_path)

#label year
census_2012['year'] = '2012'
census_2016['year'] = '2016'
census_2018['year'] = '2020'

#Join together
census_total = pd.concat([census_2012, census_2016, census_2018])

#Join Mapping file and Census data on geo_id
census_final = pd.merge(census_total, FIPS_map[['geo_id','county_name']],
                        left_on = 'geo_id',
                        right_on = 'geo_id',
                        how = 'inner')

#Extra cleaning
census_final[['County_Name_Only',
              'State_Code']] = census_final.county_name.str.split(",", 
                                                                  expand=True)
census_final['County_Name_Only'] = census_final['County_Name_Only'].str.upper()
census_final['State_Code'] = census_final['State_Code'].str.strip()

#Add prefix of bq to all columns 
census_final.columns = ["bq_" + str(col) for col in census_final.columns]

#Merge voting & age data with census data
df_harvard_bq = pd.merge(df_harvard, census_final, 
                         left_on = ['year','state_po','county'],
                         right_on = ['bq_year','bq_State_Code','bq_County_Name_Only'],
                         how = 'left')
#Clean up columns
df_harvard_bq.drop(['bq_county_name',
                    'bq_County_Name_Only',
                    'bq_State_Code',
                    'YEAR',
                    'CTYNAME',
                    'bq_do_date',
                    'bq_year'],
                     axis = 1,
                     inplace = True) 

df_harvard_bq.drop_duplicates(inplace = True)

### Sanity Check

In [12]:
print("Are any files dropped. Do counties = 1953?")
df_harvard_bq.shape

df_harvard_bq[df_harvard_bq['bq_total_pop'] < df_harvard_bq['total']]

Are any files dropped. Do counties = 1953?


,year,state,state_po,county,dem,gop,other,total,POPESTIMATE,POPEST_MALE,...,bq_speak_spanish_at_home,bq_speak_spanish_at_home_low_english,bq_pop_15_and_over,bq_pop_never_married,bq_pop_now_married,bq_pop_separated,bq_pop_widowed,bq_pop_divorced,bq_male_60_to_61,bq_male_62_to_64


#### Output File 

In [13]:
df_harvard_bq.to_csv(output_file_path, index = False)